In [1]:
cd ..

/home/hyunjun/workspace/technical_indicator


In [2]:
import talib as ta
import pandas as pd

from coinmarketcap_dataloader import CoinMarketCap
from coinmarketcap_dataloader import CMC_API_KEY

### Indicator Module

- for loop을 계속 돌리면 api error가 발생해서 time sleep을 넣음 <br>
- 전일까지의 historical data 일부는 멤버변수에 할당해놓고 사용하는 형태로 구현 <br>
- 각 지표의 기간별 값을 ticker key를 갖는 dict으로 리턴: value_dict <br>
- 각 지표의 기간별 스코어를 ticker key를 갖는 dict으로 리턴: score_dict <br> 

In [16]:
import time 
import numpy as np
import talib as ta
import multiprocessing

from tqdm import tqdm
from typing import Tuple
from datetime import datetime
from datetime import timedelta


class Indicator(CoinMarketCap):
    PERIODS = [
        7, 28, 91]

    SCORE_NAMES = [
        '7_period_score', 
        '28_period_score', 
        '91_period_score', 
        'mean_score'
        ]

    INDICATOR_NAMES = [
        'RSI', 'MACD', 
        'BBANDS', 'STOCH', 
        'OBV']


    def __init__(self, API_KEY):
        super().__init__(API_KEY)

        self.tickers:list = None
        self.historical_data_dict:dict = {}

        self.update_tickers()


    def get_total_score(self, tickers:list) -> Tuple[dict, dict]:
        """
        입력 ticker들에 대한 total score list를 리턴

        [value_dict]
        {'BTC': {'RSI': [53., 65., 62.],
                'MACD': [1317., 3107., 3222.],
                'BBANDS': [22., 88., 95.],
                'STOCH': [-2., 1., 1.],
                'OBV': [71681935600., 71681935600., 71681935600.]},

        'USDT': {'RSI': [51., 50., 50.],
                'MACD': [-5.6, 1.5, -4.],
                'BBANDS': [58., 53., 49.],
                'STOCH': [0., 0., 0.],
                'OBV': [-610460727736., -610460727736., -610460727736.]},

        'ETH': {'RSI': [50., 60., 57.],
                'MACD': [61., 138., 85.3873846637475],
                'BBANDS': [23., 85., 91.],
                'STOCH': [-1., 0., 1.],
                'OBV': [102378389016.81, 102378389016.81, 102378389016.81]}}
        

        [score_dict]
        {'BTC': {'7_period_score': 1.0,
                '28_period_score': 4.0,
                '91_period_score': 4.0,
                'mean_score': 3.0},

        'USDT': {'7_period_score': -2.0,
                '28_period_score': 0.0,
                '91_period_score': -2.0,
                'mean_score': -1.333},

        'ETH': {'7_period_score': 1.0,
                '28_period_score': 3.0,
                '91_period_score': 4.0,
                'mean_score': 2.667}}
        """

    
        tickers = set(tickers).intersection(
            set(self.historical_data_dict.keys())
        )

        indicator_func = [
            self.get_RSI,
            self.get_MACD,
            self.get_STOCH,
            self.get_BBANDS,
            self.get_OBV,
        ]

        scoring_func = [
            self.get_RSI_score,
            self.get_MACD_score,
            self.get_STOCH_score,
            self.get_BBANDS_score,
            self.get_OBV_score,
        ]

        def parrallel(tickers):
            with multiprocessing.Pool() as pool:
                value_array = list(pool.map(self.get_ALL, tickers))
            return value_array

        # 모든 ticker의 지표별, 기간별 indicator 값에 대한 어레이
        # value_array = [ self.get_ALL(ticker) for ticker in tqdm(tickers) ]
        value_array = parrallel(tickers)

        # ticker : {indicator: [values1, values2, ...]}
        value_dict = {ticker: {name:value for name, value in zip(self.INDICATOR_NAMES, values)} \
            for ticker, values in zip(tickers, value_array)}

        # 모든 ticker의 지표별, 기간별 score 값에 대한 어레이
        score_array = np.array([ [func(values) for func, values in zip(scoring_func, value_dict[ticker].values())] \
            for ticker in tickers ])

        # 모든 지표에 대해서 합하여 기간별 score 산출
        score_array_sum = np.sum(score_array, axis=1)
        # 기간별 score와 기간별 score의 평균을 묶어서 최종 score 배열로 
        score_array_total = np.concatenate([score_array_sum, np.mean(score_array_sum, axis=-1)[:,np.newaxis]], axis=1)
        score_array_total = np.round(score_array_total, decimals=3)

        # ticker: {period: score}
        score_dict = {ticker: {name:score for name, score in zip(self.SCORE_NAMES, score_list)} \
            for ticker, score_list in zip(tickers, score_array_total)}
    
        return value_dict, score_dict


    def get_ALL(self, ticker:str) -> list:
        """
        모든 지표값을 리턴
        """
        ohlcv = self._get_price_to_now(ticker, types=['open', 'high', 'low', 'close', 'volume'])
        all_indicators = [self.get_RSI, self.get_MACD, self.get_BBANDS, self.get_STOCH, self.get_OBV]
        return [func(ticker, ohlcv) for func in all_indicators]


    def get_RSI(self, ticker:str, ohlcv:pd.DataFrame = None) -> list:
        """
        Period별 RSI
        """
        data = self._get_price_to_now(ticker, ['close']) if ohlcv is None else ohlcv 
        return [ta.RSI(data['close'].to_numpy(), n)[-1] for n in self.PERIODS]

    
    def get_MACD(self, ticker:str, ohlcv:pd.DataFrame = None) -> list:
        """
        Period별 MACD 
        """
        data = self._get_price_to_now(ticker, ['close']) if ohlcv is None else ohlcv 
        return [ta.MACD(data['close'].to_numpy(), fastperiod=n, slowperiod=int((26/12)*n), signalperiod=int((9/12)*n))[0][-1] for n in self.PERIODS]


    def get_BBANDS(self, ticker:str, ohlcv:pd.DataFrame = None) -> list:
        """
        Period별 BBANDS 
        """
        data = self._get_price_to_now(ticker, ['close']) if ohlcv is None else ohlcv 

        return [(data['close'][-1] - ta.SMA(data['close'].to_numpy(), timeperiod=n)[-1]) / \
         ta.STDDEV(data['close'].to_numpy(), timeperiod=n)[-1] for n in self.PERIODS]

    
    def get_STOCH(self, ticker:str, ohlcv:pd.DataFrame = None) -> list:
        """
        Period별 STOCH
        """
        data = self._get_price_to_now(ticker, ['high', 'low', 'close']) if ohlcv is None else ohlcv 
        
        return [ta.STOCH(data['high'].to_numpy(), data['low'].to_numpy(), data['close'].to_numpy(), 
                         fastk_period=n, slowk_period=int((3/14)*n), slowd_period=int((3/14)*n))[0][-1] for n in self.PERIODS]
    

    def get_OBV(self, ticker:str, ohlcv:pd.DataFrame = None) -> list:
        """
        Period별 OBV
        """
        data = self._get_price_to_now(ticker, ['close', 'volume']) if ohlcv is None else ohlcv 

        return [ta.OBV(data['close'].to_numpy(), data['volume'].to_numpy())[-1] for n in self.PERIODS]


    def get_RSI_score(self, rsi_values:list):
        """
        RSI 지표 스코어 기준
        """
        def score_func(rsi):
            if rsi < 30: 
                return -1
            elif rsi > 70:
                return 1
            else: 
                return 0
            
        return list(map(score_func, rsi_values))
    

    def get_MACD_score(self, macd_values:list):
        """ 
        MACD 지표 스코어 기준
        """
        def score_func(macd):
            if macd > 0: 
                return 1
            else:
                return -1
            
        return list(map(score_func, macd_values))
        

    def get_BBANDS_score(self, distance_values:list):
        """
        BBANDS 지표 스코어 기준
        """
        def score_func(distance):
            if distance < -1:
                return -1
            elif distance > 1:
                return 1
            else:
                return 0
        
        return list(map(score_func, distance_values))


    def get_STOCH_score(self, slowk_values:list):
        """
        STOCH 지표 스코어 기준
        """
        def score_func(slowk):
            if slowk < 20:
                return -1
            elif slowk > 80:
                return 1
            else:
                return 0
            
        return list(map(score_func, slowk_values))
    

    def get_OBV_score(self, obv_values:list):
        """ 
        OBV 지표 스코어 기준
        """
        def score_func(obv):
            if obv > 0:
                return 1
            else:
                return -1
            
        return list(map(score_func, obv_values))


    def update_tickers(self):
        """
        Ticker List를 멤버변수 self.tickers에 저장
        """
        ticker_infos = self.get_all_listings()
        tickers = [info['symbol'] for info in ticker_infos]
        self.tickers = list( map(lambda x:x.upper(), tickers) )


    def update_historical_data_frame(self, tickers:list=None):
        """
        Ticker 과거 데이터를 멤버변수 self.historical_data_frame에 저장
        """

        tickers = self.tickers if tickers is None else tickers

        for ticker in tqdm(tickers):
            
            try: 
                # time.sleep(0.5)
                self.historical_data_dict.update({ticker:self.get_ohlcv_n(ticker)})
            except:
                try:
                    time.sleep(3.0)
                    self.historical_data_dict.update({ticker:self.get_ohlcv_n(ticker)})
                except:
                    continue


    def get_ohlcv_n(self, ticker:str, n=300) -> pd.DataFrame:
        """ 
        현재 시점으로부터 과거 n개의 ohlcv 데이터를 리턴
        """

        date_ago = datetime.now() - timedelta(days = n + 1)
        historical_info = self.get_ohlcv_historical(ticker, start=date_ago)
        dataframe = pd.DataFrame([info['quote']['USD'] for info in historical_info])
        dataframe = dataframe.set_index('timestamp', drop=True)
        return dataframe
    

    def _get_price_to_now(self, ticker:str, types:list = ['close']) -> pd.DataFrame:
        """
        과거 데이터 + 현재 시점까지의 가격 및 거래량 데이터를 리턴
        """

        # 전일까지의 price dataframe
        historical_df = self.historical_data_dict[ticker]
    
        try:
            # 현재시점의 price dataframe
            realtime_df = self.get_ohlcv_realtime([ticker])
            realtime_df = pd.DataFrame([realtime_df[ticker][0]['quote']['USD']])
            realtime_df.set_index('last_updated', inplace=True)
            # price series  
            price = pd.concat([historical_df[types], realtime_df[types]])
        
        except:
            price = historical_df[types] 

        return price
    

    def _get_normalize(self, array:np.array, upper:int=5, lower:int=-5):
        """
        Min-Max normalize으로 upper, lower bounding
        """
        k = upper - lower
        d = -lower

        new_array = array.copy()
        max_score = np.max(new_array)
        min_score = np.min(new_array)

        if len(array) == 1:
            return new_array
        
        if max_score == min_score:
            return new_array
    
        new_array = k * (new_array - min_score) / (max_score - min_score) - d
        return new_array

In [17]:
indicator = Indicator(CMC_API_KEY)

In [18]:
tickers = indicator.tickers[:50]

#### 전일까지의 과거 데이터 ticker 별로 멤버변수에 저장

In [19]:
indicator.update_historical_data_frame(tickers)

100%|██████████| 50/50 [01:20<00:00,  1.61s/it]


In [20]:
len(indicator.historical_data_dict)

50

#### 기간별 지표값 가져오기

In [75]:
indicator.get_BBANDS('ETH')

[-1.4112747020276266, 0.9040546153101282, 1.70006423128394]

In [50]:
indicator.get_RSI('ETH')

[50.60142852780077, 60.79751534106993, 57.37798574125299]

In [51]:
indicator.get_MACD('ETH')

[62.021698708510485, 138.34168173938133, 85.4144351824009]

In [52]:
indicator.get_RSI('ETH')

[50.60142852780077, 60.79751534106993, 57.37798574125299]

In [53]:
indicator.get_STOCH('ETH')

[24.963555076605946, 85.9971588008767, 91.16082106847753]

In [54]:
indicator.get_OBV('ETH')

[102331450380.84999, 102331450380.84999, 102331450380.84999]

### 기간별 score 가져오기

In [62]:
indicator.get_RSI_score(indicator.get_RSI('ETH'))

[0, 0, 0]

In [63]:
indicator.get_MACD_score(indicator.get_MACD('ETH'))

[1, 1, 1]

In [64]:
indicator.get_STOCH_score(indicator.get_STOCH('ETH'))

[0, 1, 1]

In [65]:
indicator.get_BBANDS_score(indicator.get_BBANDS('ETH'))

[-1, 0, 1]

In [66]:
indicator.get_OBV_score(indicator.get_OBV('ETH'))

[1, 1, 1]

#### ticker 여러개를 받아서 지표값, 스코어 받기

In [21]:
value_dict, score_dict = indicator.get_total_score(tickers)

In [ ]:
score_dict